In [5]:
import numpy as np
import matplotlib.pyplot as plt
from commpy.channelcoding import Trellis, conv_encode, viterbi_decode
from commpy.modulation import PSKModem
import reedsolo
from tqdm import tqdm  # For progress bar

def simulate_ber(EbN0_dB_list, num_bits_per_EbN0):
    # Convolutional code parameters
    constraint_length = 7
    generator_polynomials = np.array([171, 133])  # In octal representation
    code_rate = 1/2
    trellis = Trellis(memory=np.array([constraint_length - 1]),
                      g_matrix=generator_polynomials.reshape(1, -1))

    # Reed-Solomon code parameters
    rs_n = 255
    rs_k = 223
    rs_t = (rs_n - rs_k) // 2  # Corrects up to rs_t symbol errors
    rs_symbol_size = 8  # bits per symbol
    interleave_depth = 4  # Interleaving depth

    # Modulation parameters
    modem = PSKModem(2)  # BPSK
    bits_per_symbol = int(np.log2(modem.m))

    # BER results
    ber = []

    for EbN0_dB in EbN0_dB_list:
        # Initialize error counters
        total_bits = 0
        total_errors = 0

        # Number of bits to simulate per Eb/N0
        num_bits = num_bits_per_EbN0

        # Calculate noise variance
        EbN0_linear = 10 ** (EbN0_dB / 10)
        noise_variance = modem.Es / (2 * code_rate * EbN0_linear)

        # Generate random bits
        data_bits = np.random.randint(0, 2, num_bits)

        # Ensure data length is multiple of RS block size and interleaving depth
        bits_per_rs_block = rs_k * rs_symbol_size * interleave_depth
        num_rs_blocks = len(data_bits) // bits_per_rs_block
        data_bits = data_bits[:num_rs_blocks * bits_per_rs_block]

        # Process data in blocks
        for i in tqdm(range(num_rs_blocks), desc=f'Eb/N0={EbN0_dB} dB'):
            # Extract bits for current RS block
            start = i * bits_per_rs_block
            end = start + bits_per_rs_block
            block_bits = data_bits[start:end]

            # Reshape data for interleaving
            block_bits_reshaped = block_bits.reshape((interleave_depth, rs_k * rs_symbol_size))

            # Reed-Solomon Encoding
            rs_encoded_blocks = np.zeros((interleave_depth, rs_n), dtype=np.uint8)
            for idx, row_bits in enumerate(block_bits_reshaped):
                # Convert bits to bytes
                row_bytes = np.packbits(row_bits)
                # RS Encoding
                rs_encoder = reedsolo.RSCodec(rs_n - rs_k)
                rs_encoded = rs_encoder.encode(bytearray(row_bytes))
                # Store encoded data as numpy array
                rs_encoded_blocks[idx, :] = np.frombuffer(rs_encoded, dtype=np.uint8)

            # Interleaving
            interleaved_data = rs_encoded_blocks.T.flatten()

            # Convert interleaved bytes back to bits
            interleaved_bits = np.unpackbits(interleaved_data)

            # Convolutional Encoding
            conv_encoded_bits = conv_encode(interleaved_bits, trellis)

            # Modulation
            modulated_symbols = modem.modulate(conv_encoded_bits)

            # AWGN Channel
            noise = np.sqrt(noise_variance) * (np.random.randn(len(modulated_symbols)) + 1j * np.random.randn(len(modulated_symbols)))
            received_symbols = modulated_symbols + noise

            # Demodulation
            demodulated_bits = modem.demodulate(received_symbols, demod_type='hard')

            # Viterbi Decoding
            decoded_bits = viterbi_decode(demodulated_bits.astype(int), trellis, tb_depth=5 * (constraint_length - 1))
            decoded_bits = decoded_bits[:len(interleaved_bits)]  # Truncate to original length

            # Convert decoded bits to bytes
            decoded_bytes = np.packbits(decoded_bits)

            # Deinterleaving
            num_decoded_bytes = len(decoded_bytes)
            decoded_bytes_reshaped = decoded_bytes.reshape((rs_n, interleave_depth)).T

            # RS Decoding
            rs_decoded_blocks = []
            errors_in_block = 0
            for idx in range(interleave_depth):
                rs_received = decoded_bytes_reshaped[idx, :]
                rs_decoder = reedsolo.RSCodec(rs_n - rs_k)
                try:
                    rs_decoded = rs_decoder.decode(bytearray(rs_received))
                    rs_decoded_bits = np.unpackbits(np.frombuffer(rs_decoded[0], dtype=np.uint8))
                    rs_decoded_blocks.append(rs_decoded_bits)
                except reedsolo.ReedSolomonError:
                    # If RS decoding fails, count all bits in this RS block as errors
                    errors_in_block += rs_k * rs_symbol_size
                    rs_decoded_blocks.append(np.zeros(rs_k * rs_symbol_size, dtype=int))

            # Collect decoded bits
            rs_decoded_bits_total = np.vstack(rs_decoded_blocks).flatten()

            # Original bits for comparison
            original_bits = block_bits_reshaped.flatten()

            # Count bit errors
            bit_errors = np.sum(original_bits != rs_decoded_bits_total)
            # bit_errors += errors_in_block  # Include errors from failed RS decoding
            total_errors += bit_errors
            total_bits += len(original_bits)

        # Calculate BER
        ber_value = total_errors / total_bits
        ber.append(ber_value)
        print(f'Eb/N0={EbN0_dB} dB, BER={ber_value:.5e}')

    return EbN0_dB_list, ber

def main():
    # Parameters
    EbN0_dB_list = np.arange(8, 11, 1)
    num_bits_per_EbN0 = 2 * 10**9  # Adjust based on computational resources

    # Simulate BER
    EbN0_dB_list, ber = simulate_ber(EbN0_dB_list, num_bits_per_EbN0)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.semilogy(EbN0_dB_list, ber, 'o-', label='Simulated BER')
    plt.title('BER vs $E_b/N_0$ for GOES-R HRIT/EMWIN System')
    plt.xlabel('$E_b/N_0$ (dB)')
    plt.ylabel('Bit Error Rate (BER)')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main()


Eb/N0=8 dB:   0%|          | 104/280269 [14:28<649:58:28,  8.35s/it]


KeyboardInterrupt: 